# Packages and ngspice setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

!sudo apt-get install ngspice libngspice0

# Create a symbolic link for libngspice.so if it doesn't exist
# This is often needed because PySpice looks for libngspice.so (without the version number)
!if [ ! -f /usr/lib/x86_64-linux-gnu/libngspice.so ]; then \
    sudo ln -s /usr/lib/x86_64-linux-gnu/libngspice.so.0 /usr/lib/x86_64-linux-gnu/libngspice.so; \
    echo "Created symlink /usr/lib/x86_64-linux-gnu/libngspice.so"; \
fi

# Set the environment variable for PySpice to find ngspice
import os
os.environ['PYSPICE_NGSPICE_PATH'] = '/usr/lib/x86_64-linux-gnu/libngspice.so.0'

!pip install PySpice

# Diagnostic: Find the actual path of libngspice.so
# Please share the output of this command after execution.
!dpkg -L ngspice
!dpkg -L libngspice0

# Further Diagnostics: Check file existence and dependencies
!ls -l /usr/lib/x86_64-linux-gnu/libngspice.so
!ldd /usr/lib/x86_64-linux-gnu/libngspice.so

import PySpice.Logging.Logging as Logging
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
import re
import torch
import scipy.signal as signal
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import concurrent.futures
import multiprocessing
import random

# Netlist Processor (SPICE)

In [ ]:
class NetlistProcessor:
    """
    Motor de análisis de Netlists para el Kaspix Omni-Pipeline.
    Encargado de parsear, validar y extraer la configuración paramétrica
    de circuitos SPICE estándar.
    """

    def __init__(self, file_path):
        self.file_path = file_path
        self.content = ""
        self.params_raw = {}      # Diccionario original (texto, ej: '10k')
        self.params_float = {}    # Diccionario numérico (float, ej: 10000.0)
        self.io_config = {        # Metadatos de entrada/salida
            "input_source": None,
            "output_node": None,
            "ground_check": False
        }
        self.is_parsed = False

        # Carga inmediata
        self._load_file()

    def _load_file(self):
        if not os.path.exists(self.file_path):
            raise FileNotFoundError(f"[Kaspix Error] No se encuentra el archivo: {self.file_path}")

        with open(self.file_path, 'r', encoding='utf-8') as f:
            self.content = f.read()

    def _spice_to_float(self, value_str):
        """
        Convierte sufijos de ingeniería SPICE a float de Python.
        Soporta: T, G, Meg, k, m, u, n, p, f
        """
        value_str = value_str.lower().strip('{} ') # Limpiar llaves y espacios

        # Mapa de multiplicadores SPICE
        suffixes = {
            't': 1e12, 'g': 1e9, 'meg': 1e6, 'k': 1e3,
            'mil': 25.4e-6, 'm': 1e-3, 'u': 1e-6,
            'n': 1e-9, 'p': 1e-12, 'f': 1e-15
        }

        # Regex para separar número y sufijo
        # Busca un número (int o float) seguido opcionalmente de letras
        match = re.match(r'^([\d\.\-\+]+)([a-z]*)$', value_str)

        if not match:
            try:
                return float(value_str) # Intento directo
            except ValueError:
                return None # No es un número parseable (ej: una fórmula compleja)

        number, suffix = match.groups()
        multiplier = 1.0

        if suffix:
            # SPICE es 'greedy' con los sufijos, 'meg' es especial
            if suffix.startswith('meg'):
                multiplier = suffixes['meg']
            elif suffix[0] in suffixes:
                multiplier = suffixes[suffix[0]]

        return float(number) * multiplier

    def analyze(self):
        """
        Ejecuta el análisis léxico del netlist.
        """
        lines = self.content.splitlines()

        # Regex Compilados
        re_param = re.compile(r'\.param\s+(\w+)\s*=\s*(\{?[\w\.\-\+]+\}?)', re.IGNORECASE)
        re_source = re.compile(r'^\s*([vV]\w+)\s+(\w+)\s+(\w+)', re.IGNORECASE)
        re_save = re.compile(r'\.(?:save|print)\s+(?:v|tran)\s*\((.+?)\)', re.IGNORECASE)

        for line in lines:
            line = line.strip()
            if not line or line.startswith('*'): continue

            # 1. Detectar Parámetros (.param)
            pm = re_param.search(line)
            if pm:
                name, val_str = pm.groups()
                # Guardar raw
                self.params_raw[name] = val_str
                # Convertir a float
                val_float = self._spice_to_float(val_str)
                if val_float is not None:
                    self.params_float[name] = val_float

            # 2. Detectar Inputs (Heurística: Nombre contiene 'in' o 'src')
            sm = re_source.match(line)
            if sm:
                s_name, n_pos, n_neg = sm.groups()
                # Priorizamos fuentes que parezcan de señal
                if 'in' in s_name.lower() or 'src' in s_name.lower() or 'sig' in s_name.lower():
                    self.io_config["input_source"] = s_name

            # 3. Detectar Ground
            if ' 0 ' in line or line.endswith(' 0'):
                self.io_config["ground_check"] = True

            # 4. Detectar Output (.save)
            svm = re_save.search(line)
            if svm:
                # Extraer contenido dentro de paréntesis
                nodes = svm.group(1).replace(')', '').replace('(', '').split()
                if nodes:
                    self.io_config["output_node"] = nodes[0] # Tomamos el primero como principal

        # Fallback para Output si no hay .save
        if not self.io_config["output_node"]:
             if re.search(r'\b(out|output|salida)\b', self.content, re.IGNORECASE):
                 self.io_config["output_node"] = "out (Inferred)"
             else:
                 self.io_config["output_node"] = "UNKNOWN (Define .save V(node))"

        self.is_parsed = True
        return self.get_summary()

    def get_summary(self):
        if not self.is_parsed: return "No analizado."
        return {
            "file_path": self.file_path,
            "valid_spice": self.io_config["ground_check"],
            "knobs_detected": list(self.params_float.keys()),
            "knobs_values": self.params_float,
            "input_source": self.io_config["input_source"],
            "output_target": self.io_config["output_node"]
        }

# Dataset Generation

In [ ]:
# ============================================================
# 1. KASPIX SIGNAL FACTORY (Con Dinámica de Amplitud)
# ============================================================
class KaspixSignalFactory:
    def __init__(self, fs, duration_sec):
        self.fs = fs
        self.duration = duration_sec
        self.n_samples = int(fs * duration_sec)
        self.time_axis = np.linspace(0, duration_sec, self.n_samples)

    def _apply_dynamic_gain(self, y):
        """
        Simula la variación de volumen de entrada (Dinámica).
        """
        target_peak = np.exp(np.random.uniform(np.log(0.05), np.log(1.5)))
        max_val = np.max(np.abs(y))
        if max_val > 1e-9:
            return (y / max_val) * target_peak
        return y

    def _apply_input_noise(self, signal_in):
        """
        [NUEVO] Inyecta un 'Piso de Ruido' realista.
        Simula interferencia eléctrica, térmica o de cables.
        """
        # 1. Decidir si esta muestra tendrá ruido (90% de las veces sí)
        if np.random.rand() > 0.9:
            return signal_in

        # 2. Generar Ruido Base (Blanco)
        noise = np.random.randn(len(signal_in))

        # 3. Determinar nivel de ruido (SNR)
        # Un 'Noise Floor' típico en audio va de -60dB (bueno) a -30dB (malo/vintage)
        # Calculamos la amplitud del ruido basada en una referencia fija (no relativa a la señal)
        # Esto simula que el ruido es constante independientemente de si tocas fuerte o suave.
        noise_level_db = np.random.uniform(-70, -30)
        noise_amplitude = 10 ** (noise_level_db / 20)

        # 4. Mezclar
        noisy_signal = signal_in + (noise * noise_amplitude)

        # Opcional: Recortar picos si se pasa de +/- 1.5V (Safety Clip)
        return np.clip(noisy_signal, -1.5, 1.5)

    # --- Generadores Primitivos ---
    def _pink_noise(self):
        uneven = self.n_samples % 2
        X = np.random.randn(self.n_samples // 2 + 1 + uneven) + 1j * np.random.randn(self.n_samples // 2 + 1 + uneven)
        S = np.sqrt(np.arange(len(X)) + 1.)
        y = (np.fft.irfft(X / S)).real
        if uneven: y = y[:-1]
        # ORDEN FÍSICO: Señal -> Ganancia -> Ruido de Cable
        y = self._apply_dynamic_gain(y)
        return self._apply_input_noise(y)

    def _chirp_log(self):
        f_start = 20
        f_end = np.random.uniform(self.fs/4, self.fs/2 * 0.9)
        y = signal.chirp(self.time_axis, f0=f_start, f1=f_end, t1=self.duration, method='logarithmic')
        y = self._apply_dynamic_gain(y)
        return self._apply_input_noise(y)

    def _step_sequence(self):
        num_steps = np.random.randint(3, 12)
        y = np.zeros_like(self.time_axis)
        indices = np.sort(np.random.choice(np.arange(self.n_samples), num_steps, replace=False))
        levels = np.random.uniform(-0.8, 0.8, num_steps)
        current_idx = 0
        for i, idx in enumerate(indices):
            y[current_idx:idx] = levels[i-1] if i > 0 else 0
            current_idx = idx
        y[current_idx:] = levels[-1]
        # Los steps ya tienen amplitud propia, solo añadimos ruido
        return self._apply_input_noise(y)

    def _multitone(self):
        num_tones = np.random.randint(3, 15)
        y = np.zeros_like(self.time_axis)
        for _ in range(num_tones):
            freq = np.random.uniform(20, self.fs/3)
            phase = np.random.uniform(0, 2*np.pi)
            y += np.sin(2 * np.pi * freq * self.time_axis + phase)
        y = self._apply_dynamic_gain(y)
        return self._apply_input_noise(y)

    def _impulse_train(self):
        y = np.zeros_like(self.time_axis)
        num_impulses = np.random.randint(1, 5)
        indices = np.random.randint(0, self.n_samples, num_impulses)
        y[indices] = 1.0
        # Impulsos limpios suelen ser mejores para analisis, pero ruido leve no daña
        return self._apply_input_noise(y)

    def get_signal(self, recipe):
        keys = list(recipe.keys())
        probs = np.array(list(recipe.values()))
        probs /= probs.sum()
        choice = np.random.choice(keys, p=probs)

        sig = None
        name = "Unknown"

        if choice == 'chirp':
            sig, name = self._chirp_log(), "Chirp Log"
        elif choice == 'pink_noise':
            sig, name = self._pink_noise(), "Pink Noise"
        elif choice == 'step_sequence':
            sig, name = self._step_sequence(), "Step Seq"
        elif choice == 'multitone':
            sig, name = self._multitone(), "Multitone"
        elif choice == 'impulse':
            sig, name = self._impulse_train(), "Impulse"
        elif choice == 'sine':
            f = np.random.uniform(20, 1000)
            raw = np.sin(2*np.pi*f*self.time_axis)
            sig, name = self._apply_input_noise(self._apply_dynamic_gain(raw)), f"Sine {int(f)}Hz"
        elif choice == 'silence_decay':
             raw = np.zeros_like(self.time_axis)
             raw[:int(self.n_samples*0.1)] = np.random.randn(int(self.n_samples*0.1))
             # El silencio DEBE tener ruido de fondo para ser realista
             sig, name = self._apply_input_noise(self._apply_dynamic_gain(raw)), "Silence Decay"
        else:
            sig, name = self._pink_noise(), "Default (Pink)"

        return sig, name

# ============================================================
# 2. WORKER SIMULATOR (Con soporte para Dummy Knob)
# ============================================================
def _simulation_worker(task_payload):
    try:
        file_path = task_payload['file_path']
        input_source = task_payload['input_source']
        output_target = task_payload['output_target']
        fs = task_payload['fs']
        duration = task_payload['duration']
        input_signal = task_payload['input_signal']
        knob_config = task_payload['knob_config']
        task_id = task_payload['id']

        # --- LÓGICA SPICE ---
        parser = SpiceParser(path=file_path)
        circuit = parser.build_circuit()

        # Reconstruir eje de tiempo local
        n_samples = len(input_signal)
        time_axis = np.linspace(0, duration, n_samples)

        # Inyección de Fuente
        actual_source_name = None
        for element in circuit.element_names:
            if element.upper() == input_source.upper():
                actual_source_name = element
                break

        if actual_source_name:
            original_source = circuit[actual_source_name]
            circuit._elements.pop(actual_source_name)
            t_list = [float(x) for x in time_axis]
            v_list = [float(x) for x in input_signal]
            input_data = list(zip(t_list, v_list))
            circuit.PieceWiseLinearVoltageSource('Input_UES', original_source.nodes[0], original_source.nodes[1], values=input_data)
        else:
            return {"status": "error", "msg": f"Fuente {input_source} missing", "id": task_id}

        # Aplicar Knobs (MEJORA #3: Ignorar Dummy Knob)
        for name, val in knob_config.items():
            if name == 'dummy_param': # Si es el knob falso, no lo inyectamos en SPICE
                continue
            circuit.parameter(name, val)

        # Simular
        simulator = circuit.simulator(temperature=25, nominal_temperature=25)
        step_time = 1.0 / fs
        analysis = simulator.transient(step_time=step_time, end_time=duration)

        # Output
        output_signal = None
        if output_target in analysis.nodes:
            output_signal = np.array(analysis.nodes[output_target])
        else:
            for node_name, node_data in analysis.nodes.items():
                if str(node_name).upper() == str(output_target).upper():
                    output_signal = np.array(node_data)
                    break

        if output_signal is None:
            return {"status": "error", "msg": f"Output {output_target} missing", "id": task_id}

        if len(output_signal) != len(time_axis):
            output_signal = np.interp(time_axis, np.array(analysis.time), output_signal)

        return {
            "status": "ok",
            "id": task_id,
            "y": output_signal.astype(np.float32),
            "x_meta": {
                "knobs": np.array(list(knob_config.values()), dtype=np.float32),
                "knob_names": list(knob_config.keys())
            }
        }
    except Exception as e:
        return {"status": "error", "msg": str(e), "id": task_payload.get('id', -1)}

# ============================================================
# 3. KASPIX GENERATOR (Con inyección de Dummy Knob)
# ============================================================
class KaspixParallelGenerator:
    def __init__(self, processor_result, fs=48000, duration_sec=0.2, recipe=None, seed=42):
        self.meta = processor_result
        self.fs = fs
        self.duration = duration_sec
        self.seed = seed
        self.factory = KaspixSignalFactory(fs, duration_sec)
        self.time_axis = self.factory.time_axis
        self.recipe = recipe if recipe else {"chirp": 1.0}

        if not self.meta['valid_spice']:
            raise ValueError("[Kaspix] Netlist inválido.")

        self.max_workers = multiprocessing.cpu_count()
        print(f"⚡ Kaspix Parallel Engine V6 (Dynamic Gain + Dummy Fix): {self.max_workers} núcleos | Seed: {self.seed}")

    def _sample_knobs(self, variation_pct=0.8):
        nominal = self.meta['knobs_values']

        # [MEJORA #3] Si no hay knobs detectados, inyectar uno falso
        if not nominal:
            # Retornamos un valor fijo (ej. 1.0) para mantener la estructura FiLM
            return {'dummy_param': 1.0}

        sampled = {}
        for name, val in nominal.items():
            delta = val * variation_pct
            new_val = np.random.uniform(val - delta, val + delta)
            if new_val <= 0: new_val = val * 0.01
            sampled[name] = new_val
        return sampled

    def build_dataset(self, n_samples=100, save_path="kaspix_dataset.pt"):
        set_global_seed(self.seed)
        print(f"--- Preparando {n_samples} tareas deterministas ---")

        tasks = []
        pre_generated_signals = {}

        for i in range(n_samples):
            sig_in, sig_type = self.factory.get_signal(self.recipe)
            knobs = self._sample_knobs()

            pre_generated_signals[i] = {
                "audio_in": sig_in.astype(np.float32),
                "signal_type": sig_type
            }

            task = {
                "id": i,
                "file_path": self.meta['file_path'],
                "input_source": self.meta['input_source'],
                "output_target": self.meta['output_target'],
                "fs": self.fs,
                "duration": self.duration,
                "input_signal": sig_in,
                "knob_config": knobs
            }
            tasks.append(task)

        print(f"🚀 Procesando...")
        results_unsorted = []
        with concurrent.futures.ProcessPoolExecutor(max_workers=self.max_workers) as executor:
            results_unsorted = list(tqdm(executor.map(_simulation_worker, tasks), total=n_samples))

        print("📦 Ordenando y verificando consistencia...")
        results_sorted = sorted(results_unsorted, key=lambda x: x['id'])

        dataset_x = []
        dataset_y = []
        success_count = 0

        for res in results_sorted:
            idx = res['id']
            if res['status'] == 'ok':
                input_meta = pre_generated_signals[idx]
                worker_meta = res['x_meta']
                x_entry = {
                    "audio_in": input_meta['audio_in'],
                    "signal_type": input_meta['signal_type'],
                    "knobs": worker_meta['knobs'],
                    "knob_names": worker_meta['knob_names']
                }
                dataset_x.append(x_entry)
                dataset_y.append(res['y'])
                success_count += 1
            else:
                if success_count == 0:
                    print(f"❌ Error en muestra {idx}: {res['msg']}")

        if success_count == 0:
            print("❌ FALLO TOTAL.")
            return [], []

        print(f"💾 Guardando {success_count}/{n_samples} muestras en {save_path}...")
        torch.save({
            "x": dataset_x,
            "y": dataset_y,
            "fs": self.fs,
            "meta": self.meta,
            "recipe": self.recipe,
            "seed": self.seed
        }, save_path)

        return dataset_x, dataset_y

# Execution

In [ ]:
# ============================================================
# BLOQUE PRINCIPAL DE EJECUCIÓN (KASPIX OMNI-PIPELINE V4)
# ============================================================
if __name__ == "__main__":
    # 1. DEFINICIÓN DE OBJETIVOS Y ESTRATEGIA
    # ------------------------------------------------------------
    TARGET_NETLIST = "standard_filter_v1.cir"
    OUTPUT_DATASET = "kaspix_training_data_v4.pt"

    # Configuración de Simulación
    N_SAMPLES = 50       # Aumentamos un poco para ver variedad estadística
    SAMPLE_RATE = 48000  # Hz
    DURATION = 0.04       # Segundos

    # RECETA DE EXCITACIÓN (Domain Adaptation Strategy)
    # Definimos qué 'dieta' de señales aprenderá el modelo
    RECIPE_MIX = {
        "chirp": 0.25,          # Respuesta en Frecuencia (Bode)
        "pink_noise": 0.20,     # Dinámica Espectral Compleja
        "multitone": 0.15,      # Intermodulación
        "step_sequence": 0.20,  # Transitorios / Settling Time
        "sine": 0.10,           # Pura frecuencia
        "impulse": 0.05,        # Respuesta al Impulso
        "silence_decay": 0.05   # Piso de ruido y descarga
    }

    print(f"🚀 INICIANDO KASPIX OMNI-PIPELINE (V4 - Signal Factory)")
    print(f"📂 Objetivo Físico: {TARGET_NETLIST}")
    print(f"Fn Receta de Señales: {RECIPE_MIX}")

    # 2. FASE 1: ANÁLISIS DE TOPOLOGÍA (NetlistProcessor)
    # ------------------------------------------------------------
    print("\n[Fase 1] Analizando Física del Netlist...")
    # Asegurarse de tener el archivo (crearlo si no existe para la demo)
    if not os.path.exists(TARGET_NETLIST):
        print("⚠️ ERROR No hay archivo .cir")

    processor = NetlistProcessor(TARGET_NETLIST)
    circuit_meta = processor.analyze()

    # Validación Estricta
    if not circuit_meta['valid_spice']:
        print("❌ CRITICAL: El Netlist no es válido (GND 0 missing?).")
        exit(1)

    print("✅ Netlist Interpretado Correctamente.")
    print(f"   -> Knobs: {list(circuit_meta['knobs_values'].keys())}")
    print(f"   -> I/O: {circuit_meta['input_source']} -> {circuit_meta['output_target']}")

    # 3. FASE 2: GENERACIÓN MASIVA (KaspixParallelGenerator)
    # ------------------------------------------------------------
    print(f"\n[Fase 2] Generando {N_SAMPLES} simulaciones con Variabilidad Estocástica...")

    # Instanciamos con la RECETA
    generator = KaspixParallelGenerator(
        processor_result=circuit_meta,
        fs=SAMPLE_RATE,
        duration_sec=DURATION,
        recipe=RECIPE_MIX  # <--- Inyección de Estrategia
    )

    data_x, data_y = generator.build_dataset(
        n_samples=N_SAMPLES,
        save_path=OUTPUT_DATASET
    )

    # 4. FASE 3: CONTROL DE CALIDAD VISUAL (QC)
    # ------------------------------------------------------------
    # Si generamos 0 muestras por error, evitar crash
    if len(data_x) == 0:
        print("❌ Error: No se generaron datos.")
        exit(1)

    print("\n[Fase 3] Control de Calidad Visual (Inputs Diversos)...")

    # --- FIX: Generar eje de tiempo manualmente ---
    # Como el generador paralelo no expuso time_axis, lo calculamos aquí
    # usando los parámetros globales definidos al inicio.
    time_axis = np.linspace(0, DURATION, int(SAMPLE_RATE * DURATION))
    # ----------------------------------------------

    indices_to_plot = [0, N_SAMPLES // 2, N_SAMPLES - 1]
    indices_to_plot = [i for i in indices_to_plot if i < len(data_x)]

    plt.figure(figsize=(12, 10))

    # Subplot 1: Señales de Excitación
    ax1 = plt.subplot(2, 1, 1)
    ax1.set_title("Inputs Generados (Signal Factory)")

    for idx in indices_to_plot:
        sig_type = data_x[idx]['signal_type']
        ax1.plot(
            time_axis * 1000,    # <--- USAMOS LA VARIABLE LOCAL
            data_x[idx]['audio_in'],
            alpha=0.7,
            label=f"#{idx}: {sig_type}"
        )
    ax1.set_ylabel("Voltaje (V)")
    ax1.legend(loc='upper right', fontsize='small')
    ax1.grid(True, alpha=0.3)

    # Subplot 2: Respuestas Físicas
    ax2 = plt.subplot(2, 1, 2)
    # Safely get knobs for title
    sample_knobs = list(data_x[0]['knob_names']) if data_x else []
    ax2.set_title(f"Respuestas del Sistema (Knobs: {sample_knobs})")

    for idx in indices_to_plot:
        knob_vals = data_x[idx]['knobs']
        sig_type = data_x[idx]['signal_type']
        k_names = data_x[idx]['knob_names']

        params_str = " | ".join([f"{n}:{v:.1e}" for n, v in zip(k_names, knob_vals)])

        ax2.plot(
            time_axis * 1000,   # <--- USAMOS LA VARIABLE LOCAL
            data_y[idx],
            label=f"#{idx} [{sig_type}]: {params_str}"
        )

    ax2.set_xlabel("Tiempo (ms)")
    ax2.set_ylabel("Voltaje (V)")
    ax2.legend(loc='upper right', fontsize='x-small')
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"\n🎉 KASPIX PIPELINE COMPLETADO.")
    print(f"   Dataset guardado en: {OUTPUT_DATASET}")

# How to load data (FiLM) and Parameter Statistics

In [ ]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

class KaspixDatasetFiLM(Dataset):
    def __init__(self, pt_file_path):
        print(f"📂 Cargando Dataset FiLM desde: {pt_file_path}")

        # --- FIX 1: weights_only=False ---
        # Necesario porque nuestro .pt contiene diccionarios, listas y numpy arrays,
        # no solo pesos de modelo.
        loaded = torch.load(pt_file_path, weights_only=False)

        self.data_x = loaded['x']
        self.data_y = loaded['y']
        self.meta = loaded['meta']

        # Calcular estadísticas para normalizar Knobs
        all_knobs = np.array([item['knobs'] for item in self.data_x])
        self.k_min = torch.from_numpy(all_knobs.min(axis=0)).float()
        self.k_max = torch.from_numpy(all_knobs.max(axis=0)).float()

        # Evitar división por cero
        self.k_max[self.k_max == self.k_min] += 1e-6

        self.knob_names = self.data_x[0]['knob_names'] # Guardamos nombres para debug

        print(f"✅ Dataset Cargado: {len(self.data_x)} muestras.")
        print(f"   -> Params Normalization ({len(self.k_min)} vars): {self.knob_names}")

    def __len__(self):
        return len(self.data_x)

    def __getitem__(self, idx):
        # A. Audio Input [1, L]
        audio_raw = self.data_x[idx]['audio_in']
        x_audio = torch.from_numpy(audio_raw).float().unsqueeze(0)

        # B. Knobs Input [N]
        knobs_raw = self.data_x[idx]['knobs']
        x_knobs = torch.from_numpy(knobs_raw).float()
        x_knobs_norm = (x_knobs - self.k_min) / (self.k_max - self.k_min)

        # C. Target [1, L]
        target_raw = self.data_y[idx]
        y_target = torch.from_numpy(target_raw).float().unsqueeze(0)

        return x_audio, x_knobs_norm, y_target

# ============================================================
# BLOQUE DE PRUEBA: VERIFICAR TENSORES (ADAPTADO A FILM)
# ============================================================
if __name__ == "__main__":
    # Cargar Dataset
    ds = KaspixDatasetFiLM("kaspix_training_data_v4.pt")

    # DataLoader
    dl = DataLoader(ds, batch_size=4, shuffle=True)

    # --- FIX 2: Desempaquetar 3 valores, no 2 ---
    # Ahora recibimos (Audio, Knobs, Target)
    b_audio, b_knobs, b_target = next(iter(dl))

    print("\n--- INSPECCIÓN DE TENSOR (FiLM Architecture) ---")
    print(f"1. Audio Input Shape:  {b_audio.shape}")   # (Batch, 1, Time)
    print(f"2. Knobs Input Shape:  {b_knobs.shape}")   # (Batch, N_Params)
    print(f"3. Target Shape:       {b_target.shape}")  # (Batch, 1, Time)

    # Verificación de datos
    print(f"\nEjemplo Muestra #0:")
    print(f"  Knobs ({ds.knob_names}): {b_knobs[0].numpy()}")

    # Check de rangos
    if torch.max(b_knobs) <= 1.0 + 1e-5 and torch.min(b_knobs) >= 0.0 - 1e-5:
        print("\n✅ NORMALIZACIÓN EXITOSA: Knobs en rango [0, 1].")
        print("🚀 DATOS LISTOS PARA ARQUITECTURA FiLM.")
    else:
        print(f"\n⚠️ ALERTA: Valores fuera de rango. Min: {torch.min(b_knobs)}, Max: {torch.max(b_knobs)}")